####Converting dataset volumes in tables into bronze using streaming and DLT to make NRT flow (Ingest raw data into bronze -still raw-)


In [0]:
from pyspark.sql.functions import *

In [0]:
volume_path = "/Volumes/well_logs_nrt/landing/json_files_volume/"
schema_path = "/Volumes/well_logs_nrt/autoloader/schemas_volume/all_wells_bronze_schema"
checkpoints_path = "/Volumes/well_logs_nrt/autoloader/checkpoints_volume/all_wells_bronze_checkpoints"
output_path = "/Volumes/well_logs_nrt/bronze/deltalake_folders_volume/all_wells_bronze"

df = (spark.readStream.format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.inferColumnTypes", "true")
      .option("cloudFiles.schemaLocation", schema_path)
      .option("cloudFiles.schemaEvolutionMode", "addNewColumns") # In case that appears new columns in the log
      .load(volume_path)
      .withColumn("bronze_ingestion_time", current_timestamp())
      .withColumn("file_mod_time", col("_metadata.file_modification_time")) # useful for audit
)

df.writeStream.format("delta") \
  .option("checkpointLocation", checkpoints_path) \
  .option("mergeSchema", "true") \
  .outputMode("append") \
  .trigger(once=True) \
  .start(output_path)

In [0]:
display(spark.read.format("delta").load("/Volumes/well_logs_nrt/bronze/deltalake_folders_volume/all_wells_bronze"))